In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

import plotly.io as pio
pio.renderers.default = "browser"
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "iframe_connected"

In [5]:
df = pd.read_csv("NCH_ambulatory_appointments.csv")

df["appointment_date"] = pd.to_datetime(df["appointment_date"])
df["request_date"] = pd.to_datetime(df["scheduled_date"])

df["wait_days"] = (df["appointment_date"] - df["request_date"]).dt.days


df["within_14"] = df["wait_days"] <= 14
df["no_show_flag"] = df["status"] == "No-show"
df["cancel_flag"] = df["status"].isin([
    "Cancelled by Patient",
    "Cancelled by Clinic"
])


specialty_table = df.groupby("specialty").agg(
    total_appointments=("specialty", "count"),
    avg_wait_days=("wait_days", "mean"),
    median_wait_days=("wait_days", "median"),
    pct_within_14=("within_14", "mean"),
    pct_no_show=("no_show_flag", "mean"),
    pct_cancelled=("cancel_flag", "mean")
).reset_index()

specialty_table[[
    "pct_within_14", "pct_no_show", "pct_cancelled"
]] *= 100


fig1 = px.bar(
    specialty_table.sort_values("pct_within_14", ascending=False),
    x="specialty",
    y="pct_within_14",
    title="Percent of Patients Seen Within 14 Days by Specialty",
    color="specialty"
)

fig1.add_hline(
    y=85,
    line_dash="dash",
    line_color="red",
    annotation_text="85% Target",
    annotation_position="top left"
)


fig2 = px.box(
    df,
    x="specialty",
    y="wait_days",
    color="specialty",
    title="Wait Time Distribution by Specialty"
)


fig3 = go.Figure(
    data=[go.Table(
        header=dict(
            values=list(specialty_table.columns),
            fill_color='lightgrey',
            align='left'
        ),
        cells=dict(
            values=[specialty_table[col] for col in specialty_table.columns],
            align='left'
        )
    )]
)

fig3.update_layout(title="Specialty Access Metrics Summary")



fig1.show()
fig2.show()
fig3.show()
